In [ ]:
import numpy as np
import pandas as pd
import os
import tkinter
from tkinter import *
from tkinter.ttk import *
from tkinter.filedialog import askdirectory, askopenfilename, asksaveasfilename
import Bio
from Bio import SeqIO
from Bio import Entrez
from Bio import Phylo
from Bio import AlignIO
import random

In [ ]:
def popup_request_fasta():    
    root = tkinter.Tk()
    request_load_file = askopenfilename(initialdir=os.getcwd(),title="Select FASTA")
    if request_load_file:
        filepath = request_load_file
    else: 
        filepath='No File Selected'
    root.destroy()
    return filepath

def popup_request_savefile():    
    root = tkinter.Tk()
    request_load_file = asksaveasfilename(initialdir=os.getcwd(),title="Save File")
    if request_load_file:
        filepath = request_load_file
    else: 
        filepath='No Directory Selected'
    root.destroy()
    return filepath

def grab_defline():
    with open(popup_request_fasta()) as f:
        lines = f.readlines()
    templist=[]
    for l in lines:
        if l[0]=='>':
            templist.append(l)
    return templist

def get_accession_codes(fasta):    
    accession_list=[]
    for i in fasta:
        accession_list.append(i[1:].split()[0])
    return accession_list

def create_fasta(accession_list):
        fasta_string = str()
        fasta_list = list()
        saveloc = popup_request_savefile()
        for accession in accession_list:
            try:
                Entrez.email = 'chad.buteo@gmail.com'
                handle = Entrez.efetch(db="protein", id=accession, rettype="gb", retmode="text")
                record = SeqIO.read(handle, "genbank")
                handle.close()
                fasta_list.append(SeqIO.FastaIO.as_fasta(record))
            except Exception:
                print(f'Exception for {accession}')
            with open(saveloc, "w") as text_file:
                text_file.write(fasta_string.join(fasta_list))
        print('check for new fasta file in selected directory')

def pair_description_to_accession(combined_fasta):
    templist = []
    for id in combined_fasta:
        start = id.find('[')+1
        end = id.find(']')
        species = id[start:end]
        accession = id[1:].split()[0]
        templist.append((accession,species))
    return templist

def replace_deflines(fasta_lines):
    templines = fasta_lines.copy()
    for l in templines:
        if l in deflines:
            accession = l[1:].split()[0]
            species = '_'.join(accession_dict[accession].split())
            templines[templines.index(l)] = '>'+accession+'__'+species+'\n'
            #templines.replace(l,accession_dict[accession])
        else: pass
    return templines

def write_fasta(fasta):
    fasta_string = str()
    saveloc = popup_request_savefile()
    with open(saveloc, "w") as text_file:
        text_file.write(fasta_string.join(fasta))
    print('check for new fasta file in selected directory')

In [ ]:
combined_fasta=[]

In [ ]:
# choose the first fasta file you want to combine.... repeat as needed

for i in grab_defline():
    combined_fasta.append(i)

In [ ]:
len(combined_fasta)

In [ ]:
print(f'Number of replicate sequences: {len(combined_fasta)-len(set(combined_fasta))}')

In [ ]:
combined_fasta=list(set(combined_fasta))
combined_fasta

In [ ]:
get_accession_codes(combined_fasta)

In [ ]:
accession_list=get_accession_codes(combined_fasta)

In [ ]:
eighty_percent=round(len(accession_list)*0.8)
train_set=random.sample(accession_list,eighty_percent)
train_set

In [ ]:
len(train_set)

In [ ]:
#create_fasta(accession_list)

In [ ]:
create_fasta(train_set)

In [ ]:
test_set=[i for i in accession_list if i not in train_set]

In [ ]:
create_fasta(test_set)

In [ ]:
accession_tuples = pair_description_to_accession(combined_fasta)
#accession_tuples

In [ ]:
accession_dict = dict(accession_tuples)
accession_dict

In [ ]:
# choose alignment file
with open(popup_request_fasta(),'r') as f:
    lines = f.readlines()
#lines

In [ ]:
# select the same alignement file, I will streamline this later

deflines = grab_defline()
deflines

In [ ]:
edited_fasta = replace_deflines(lines)

In [ ]:
write_fasta(edited_fasta)